In [69]:
%env NX_CURGAPH_AUTOCONFIG=True
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

env: NX_CURGAPH_AUTOCONFIG=True


# 数据处理

In [70]:
edges_all = pd.read_csv('data/edges_all.csv')

C:\Users\wzh20\AppData\Local\Temp\ipykernel_10976\1467553205.py:1: DtypeWarning: Columns (15,19) have mixed types. Specify dtype option on import or set low_memory=False.
  edges_all = pd.read_csv('data/edges_all.csv')


In [71]:
edges_all["lanes"] = pd.to_numeric(edges_all["lanes"], errors="coerce")
edges_all["maxspeed"] = edges_all["maxspeed"].apply(
    lambda x: x.replace("mph", "").strip() if isinstance(x, str) else x
)
# 如果 'maxspeed' 列包含像 ['45 ', '60 '] 这样的列表，取出第一个值
edges_all["maxspeed"] = edges_all["maxspeed"].apply(
    lambda x: eval(x)[0] if isinstance(x, str) and x.startswith("[") else x
)
edges_all["maxspeed"] = pd.to_numeric(edges_all["maxspeed"], errors="coerce")
edges_all["lanes"].fillna(edges_all["lanes"].median(), inplace=True)
edges_all["maxspeed"].fillna(edges_all["maxspeed"].median(), inplace=True)
traffic_data["AADT 2022"] = pd.to_numeric(traffic_data["AADT 2022"], errors="coerce")
traffic_data["AADT 2022"].fillna(traffic_data["AADT 2022"].median(), inplace=True)
edges_all = edges_all.merge(
    traffic_data[["Road Name", "AADT 2022"]],
    left_on="name",
    right_on="Road Name",
    how="left",
)

edges_all["AADT 2022"].fillna(edges_all["AADT 2022"].median(), inplace=True)


KeyboardInterrupt: 

In [60]:
nodes_all = pd.read_csv('data/nodes_all.csv')
pos=dict()
for _,row in nodes_all.iterrows():
    pos[row['osmid']]=(row['x'],row['y'])

C:\Users\wzh20\AppData\Local\Temp\ipykernel_10976\8115742.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  nodes_all = pd.read_csv('data/nodes_all.csv')


# 绘图

In [68]:
alpha, beta, gamma = 0.5, 0.4, 0.5


def calculate_weight(row):
    length_weight = row["length"] / max_length
    aadt_weight = 0
    #aadt_weight = row["AADT 2022"] / max_aadt
    lanes_weight = row["lanes"] / max_lanes
    return alpha * length_weight + beta * aadt_weight + gamma * lanes_weight

max_length = edges_all["length"].max()
max_aadt = edges_all["AADT 2022"].max()
max_lanes = edges_all["lanes"].max()
for _, row in edges_all.iterrows():
    edges_all.loc[_, "weight"] = calculate_weight(row)

KeyboardInterrupt: 

In [65]:
G = nx.DiGraph()
cnt=0
for _, row in edges_all.iterrows():
    if row["u"] == row["v"]:
        continue
    cnt+=1
    if cnt>10000:
        break
    G.add_edge(
        row["u"],
        row["v"],
        weight=row["weight"],
        length=row["length"],
        lanes=row["lanes"],
        maxspeed=row["maxspeed"],
    )

In [ ]:
from tqdm import tqdm 

plt.figure(figsize=(100,100))
print("Drawing nodes")
#nx.draw_networkx_nodes(G,pos,node_size=1,node_color='blue',alpha=0.3)
print("Drawing edges")
nx.draw_networkx_edges(G,pos,edge_color='gray',width=3,style='dashed')
print("Drawing high weight edges")
high_weight_edges = [(u, v) for u, v, d in G.edges(data=True) if d['weight'] > 0.15]
nx.draw_networkx_edges(G,pos,edgelist=high_weight_edges,edge_color='red',width=5,style='solid') 

plt.title('Traffic Netword of Baltimore',fontsize=16)
plt.axis('off')
plt.show()

Drawing nodes
Drawing edges
Drawing high weight edges


In [79]:
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

In [ ]:
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G, seed=42)

nx.draw_networkx_nodes(G, pos, node_size=20, node_color='blue', alpha=0.7)
nx.draw_networkx_edges(G, pos, edge_color='gray', alpha=0.5)

high_weight_edges = [(u, v) for u, v, d in tqdm(G.edges(data=True), desc="Collecting high weight edges", unit="edge") if d.get('weight', 0) > 0.8]

def draw_high_weight_edge(u, v):
    nx.draw_networkx_edges(G, pos, edgelist=[(u, v)], edge_color='red', width=2)

with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(lambda edge: draw_high_weight_edge(edge[0], edge[1]), high_weight_edges), desc="Drawing high weight edges", total=len(high_weight_edges), unit="edge"))

plt.title('Traffic Network of Baltimore', fontsize=16)
plt.axis('off')  
plt.show()

KeyboardInterrupt: 

<Figure size 1200x800 with 0 Axes>

In [ ]:
def compute_shortest_payh(graph,source,target):
    return nx.shortest_path(graph,source=source,target=target,weight='weight')

In [ ]:
try:
    shortest_path = compute_shortest_payh(G,source=27029857,target=775383890)
    print("Shortest path(by weight):"shortest_path)
except nx.NetworkXNoPath:
    print("No path exists between the selected nodes.")

In [ ]:
average_degree = np.mean([deg for _,deg in G.degree])
print("Average Degree of the Netword:",average_degree)

In [ ]:
edges_all.to_csv("processed_edges_all.csv",index=False)